In [7]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml import Input,Output, automl, command
from azure.ai.ml.constants import AssetTypes

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: .\config.json


In [4]:
prepare_data = load_component(source="split_info.yaml")

In [94]:
logistic_regression = load_component(source="logistic_regression.yaml")

In [95]:
path_input = "azureml://subscriptions/3de29edd-5970-4662-b0fb-929561c50c41/resourcegroups/riskanalyzers-rg/workspaces/riskanalyzers-mlw/datastores/workspaceblobstore/paths/UI/2024-05-25_232652_UTC/UCI_Credit_Card.csv"

In [96]:
@pipeline()
def training_pipeline(file: Input):
    # Paso 1: Preparar los datos utilizando el componente de división de datos
    prepare_data_step = prepare_data(input_data=file)
    
    # Paso 2: Entrenar el modelo utilizando el componente de regresión logística
    train_model_step = logistic_regression(input_dir = prepare_data_step.outputs.output_path)
    
    return {
        "output_dir": prepare_data_step.outputs.output_path,
        "output_preds": train_model_step.outputs.predictions
    }

In [19]:
# Define pipeline
@pipeline(
    description="AutoML Classification Pipeline",
    )
def automl_classification(
    classification_train_data,
    classification_validation_data
):
    # define the automl classification task with automl function
    classification_node = automl.classification(
        training_data=classification_train_data,
        validation_data=classification_validation_data,
        target_column_name="default",
        primary_metric="accuracy",
        # currently need to specify outputs "mlflow_model" explictly to reference it in following nodes 
        outputs={"best_model": Output(type="mlflow_model")},
    )
    # set limits and training
    classification_node.set_limits(max_trials=1)
    classification_node.set_training(
        enable_stack_ensemble=False,
        enable_vote_ensemble=False,
        allowed_training_algorithms=['logistic_regression','light_gbm','random_forest']
    )

    command_func = command(
        inputs=dict(
            automl_output=Input(type="mlflow_model")
        ),
        command="ls ${{inputs.automl_output}}",
        environment="azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest"
    )
    show_output = command_func(automl_output=classification_node.outputs.best_model)

In [21]:
URI_TRAIN_PATH = "azureml://subscriptions/3de29edd-5970-4662-b0fb-929561c50c41/resourcegroups/riskanalyzers-rg/workspaces/riskanalyzers-mlw/datastores/workspaceblobstore/paths/LocalUpload/997b08903632ae607733d8b3c3ab8345/train_data/"
URI_TEST_PATH = "azureml://subscriptions/3de29edd-5970-4662-b0fb-929561c50c41/resourcegroups/riskanalyzers-rg/workspaces/riskanalyzers-mlw/datastores/workspaceblobstore/paths/LocalUpload/7349a5f9f32a1e5df860eeaa80bc1a31/test_data/"

In [22]:
import logging
try:
    pipeline_job = automl_classification(
        classification_train_data=Input(path=URI_TRAIN_PATH, type=AssetTypes.MLTABLE),
        classification_validation_data=Input(path=URI_TEST_PATH, type=AssetTypes.MLTABLE),
    )
    
    # Set pipeline level compute
    pipeline_job.settings.default_compute = "juan-risk"

    # Submit the pipeline job
    returned_pipeline_job = ml_client.jobs.create_or_update(
        pipeline_job,
        experiment_name="project-codigo-facilito"
    )

    logging.info("Pipeline job submitted successfully: %s", returned_pipeline_job)
except Exception as e:
    logging.error("An error occurred while creating or updating the pipeline job: %s", e)
    raise

In [97]:
# Crear y enviar la canalización
pipeline_job = training_pipeline(file=Input(type=AssetTypes.URI_FILE, path=path_input))

In [98]:
# set pipeline level compute
pipeline_job.settings.default_compute = "juan-risk"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

In [99]:
print(pipeline_job)

display_name: training_pipeline
type: pipeline
inputs:
  file:
    type: uri_file
    path: azureml://subscriptions/3de29edd-5970-4662-b0fb-929561c50c41/resourcegroups/riskanalyzers-rg/workspaces/riskanalyzers-mlw/datastores/workspaceblobstore/paths/UI/2024-05-25_232652_UTC/UCI_Credit_Card.csv
outputs:
  output_dir:
    type: uri_folder
  output_preds:
    type: uri_folder
jobs:
  prepare_data_step:
    type: command
    inputs:
      input_data:
        path: ${{parent.inputs.file}}
    outputs:
      output_path: ${{parent.outputs.output_dir}}
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: split_data
      version: '2'
      display_name: Split data
      type: command
      inputs:
        input_data:
          type: uri_file
      outputs:
        output_path:
          type: uri_folder
      command: python split_info.py --input_data ${{inputs.input_data}} --output_path
        ${{outputs.output_path}}
      envir

In [100]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="training_pipeline"
)

Uploading pipeline (2.88 MBs): 100%|##########| 2876024/2876024 [00:01<00:00, 1676956.36it/s]


